In [1]:
#Neat notebook for finding core reactions in Human1 model.
import cobra
import matplotlib.pyplot as plt
import gurobipy as gp
import pandas as pd
from pandas import DataFrame

In [2]:
#Upload Human1 model.
Human1 = "Human-GEM.xml"
model = cobra.io.read_sbml_model(Human1)
model

Academic license - for non-commercial use only - expires 2022-07-28
Using license file /Users/katemeeson/gurobi.lic


Name,HumanGEM
Memory address,0x0117c70d30
Number of metabolites,8400
Number of reactions,13096
Number of groups,143
Objective expression,1.0*biomass_human - 1.0*biomass_human_reverse_fb2f2
Compartments,"Cytosol, Extracellular, Lysosome, Endoplasmic reticulum, Mitochondria, Peroxisome, Golgi apparatus, Nucleus, Inner mitochondria"


In [3]:
#Check we can access the model's properties properly. 
print("Reactions:", len(model.reactions))
print("Metabolites:", len(model.metabolites))
print("Genes:", len(model.genes))

Reactions: 13096
Metabolites: 8400
Genes: 3628


In [4]:
#Finding biomass reactions, so I can pick objective function. 
for r in model.reactions:
    if "biomass" in r.name:
        print(r.id, ': ', r.name)

HMR_10062 :  Protein pool for biomass reaction
HMR_10063 :  Lipid pool for biomass reaction
HMR_10064 :  Small metabolite pool for biomass reaction
HMR_10065 :  Cofactors and vitamins pool for biomass reaction
biomass_human :  Generic human cell biomass reaction


In [5]:
#Running FBA, with biomass_human as objective function. 
biomass_rxn = model.reactions.get_by_id("biomass_human")
biomass_rxn.objective_coefficient = 1
biomass_rxn = model.reactions.get_by_id("HMR_10062")
biomass_rxn.objective_coefficient = 0
biomass_rxn = model.reactions.get_by_id("HMR_10063")
biomass_rxn.objective_coefficient = 0
biomass_rxn = model.reactions.get_by_id("HMR_10064")
biomass_rxn.objective_coefficient = 0
biomass_rxn = model.reactions.get_by_id("HMR_10065")
biomass_rxn.objective_coefficient = 0
solution = model.optimize()
print("FBA status:", solution.status)
print("FBA solution:", solution.objective_value)

FBA status: optimal
FBA solution: 187.35362997658078


In [6]:
#Finding number of core reactions (reactions where flux is not zero, i.e. below or above zero).
core_reactions = []
all_other_reactions = []

for r in model.reactions:
    if solution.fluxes[r.id]>0:
        core_reactions.append(r.id)
        continue
    if solution.fluxes[r.id]<0:
        core_reactions.append(r.id)
        continue
    else:
        all_other_reactions.append(r.id)
        continue 
            
print(('Amount of core reactions: '),(len(core_reactions)))
print(('Head of core reactions: '),(core_reactions[0:5]))
print('\n')
print(('Other reactions (zero flux): '),(len(all_other_reactions)))
print(('Head of all other reactions: '),(all_other_reactions[0:5]))

Amount of core reactions:  838
Head of core reactions:  ['HMR_4355', 'HMR_4363', 'HMR_4365', 'HMR_4368', 'HMR_4371']


Other reactions (zero flux):  12258
Head of all other reactions:  ['HMR_3905', 'HMR_3907', 'HMR_4097', 'HMR_4099', 'HMR_4108']


In [7]:
#Converting core reactions list into DF so it can be added as an extra column to Human1 CSV, ready for model filtering. 
core_reactionDF = pd.DataFrame(core_reactions)
core_reactionDF.to_csv(r'/Users/katemeeson/Desktop/core_reaction_ids.csv', index = False, header=True)

In [8]:
#Upload CSV with core reactions added as new column. Now filter Human1 model to just show core reactions. 
filtering_model = pd.read_csv('/Users/katemeeson/Dropbox (The University of Manchester)/filtering_human1_for_core_reactions.csv')
filtering_model

,CoreReactions,ReactionID,UniprotID,ReactionRule
0,HMR_4355,HMR_3905,"HOT_HUMAN', 'ADH1G_HUMAN', 'ADHX_HUMAN', 'PTGR...",Q8IWW8 or P28332 or Q8N4Q0 or P07327 or P40394...
1,HMR_4363,HMR_3907,AK1A1_HUMAN',P14550
2,HMR_4365,HMR_4097,ACSA_HUMAN',Q9NR19
3,HMR_4368,HMR_4099,"ACS2L_HUMAN', 'ACSS3_HUMAN'",Q9H6R3 or Q9NUB1
4,HMR_4371,HMR_4108,ACSA_HUMAN',Q9NR19
...,...,...,...,...
13091,NaN,HMR_10128,NaN,NaN
13092,NaN,HMR_10129,NaN,NaN
13093,NaN,HMR_10130,NaN,NaN
13094,NaN,HMR_10131,NaN,NaN


In [12]:
#Assigning series names to CoreReactions and ReactionID column. 
Core_reactions = filtering_model.iloc[:,0]
ReactionIDs = filtering_model.iloc[:,1]

In [15]:
filtered_model = []

for i in range(13096): #number of Human1 reactions to search. 
    for x in range(840): #series of core reactions to filter model against. 
        if ReactionIDs[i]==Core_reactions[x]:
            print(filtering_model.iloc[i,1:4])
            print('\n')
            filtered_model.append(filtering_model.iloc[i,1:4])
        else:
            continue

ReactionID                                        HMR_4355
UniprotID       ALDOA_HUMAN', 'ALDOB_HUMAN', 'ALDOC_HUMAN'
ReactionRule          P09972 or P05062 or P04075 or P04075
Name: 13, dtype: object


ReactionID                                     HMR_4363
UniprotID       ENOA_HUMAN', 'ENOB_HUMAN', 'ENOG_HUMAN'
ReactionRule                 P06733 or P13929 or P09104
Name: 16, dtype: object


ReactionID                                               HMR_4365
UniprotID       PMGE_HUMAN', 'PGAM1_HUMAN', 'PGAM4_HUMAN', 'PG...
ReactionRule                 P15259 or P18669 or P07738 or Q8N0Y7
Name: 17, dtype: object


ReactionID                                               HMR_4368
UniprotID       PGK2_HUMAN', 'CRIS3_HUMAN', 'PGK1_HUMAN', 'TGO...
ReactionRule                 P54108 or P00558 or Q5JRA6 or P07205
Name: 18, dtype: object


ReactionID                                               HMR_4371
UniprotID       PMGE_HUMAN', 'PGAM1_HUMAN', 'PGAM4_HUMAN', 'PG...
ReactionRule           

ReactionID                                        HMR_3931
UniprotID       TRXR3_HUMAN', 'TRXR2_HUMAN', 'TRXR1_HUMAN'
ReactionRule                    Q9NNW7 or Q86VQ6 or Q16881
Name: 327, dtype: object


ReactionID                       HMR_4210
UniprotID       KPYR_HUMAN', 'KPYM_HUMAN'
ReactionRule             P14618 or P30613
Name: 345, dtype: object


ReactionID         HMR_4470
UniprotID       DPYD_HUMAN'
ReactionRule         Q12882
Name: 350, dtype: object


ReactionID                                               HMR_4484
UniprotID       RBAK_HUMAN', '5NTD_HUMAN', 'NT5M_HUMAN', '5NT3...
ReactionRule    P15309 or P49902 or Q9BXI3 or Q9H0P0 or Q8TCD5...
Name: 353, dtype: object


ReactionID                       HMR_4485
UniprotID       PNPH_HUMAN', 'TYPH_HUMAN'
ReactionRule             P19971 or P00491
Name: 354, dtype: object


ReactionID        HMR_4512
UniprotID       KCY_HUMAN'
ReactionRule        P30085
Name: 356, dtype: object


ReactionID         HMR_4577
UniprotID       UM

ReactionID         HMR_8787
UniprotID       DCMC_HUMAN'
ReactionRule         O95822
Name: 817, dtype: object


ReactionID                                     HMR_6725
UniprotID       ATTY_HUMAN', 'AATM_HUMAN', 'AATC_HUMAN'
ReactionRule                 P17174 or P00505 or P17735
Name: 856, dtype: object


ReactionID      HMR_6875
UniprotID            NaN
ReactionRule         NaN
Name: 960, dtype: object


ReactionID          HMR_6888
UniprotID       TYRP2_HUMAN'
ReactionRule          P40126
Name: 971, dtype: object


ReactionID                                       HMR_3877
UniprotID       SAHH3_HUMAN', 'SAHH_HUMAN', 'SAHH2_HUMAN'
ReactionRule                   P23526 or Q96HN2 or O43865
Name: 980, dtype: object


ReactionID                        HMR_4158
UniprotID       UAP1_HUMAN', 'UAP1L_HUMAN'
ReactionRule              Q16222 or Q3KQV9
Name: 1091, dtype: object


ReactionID                         HMR_4300
UniprotID       GFPT2_HUMAN', 'GFPT1_HUMAN'
ReactionRule               O9480

ReactionID                                     HMR_3144
UniprotID       HCDH_HUMAN', 'HCD2_HUMAN', 'ECHA_HUMAN'
ReactionRule               Q99714 and P40939 and Q16836
Name: 2704, dtype: object


ReactionID                                     HMR_3146
UniprotID       ECHB_HUMAN', 'THIM_HUMAN', 'ECHA_HUMAN'
ReactionRule               P40939 and P55084 and P42765
Name: 2705, dtype: object


ReactionID                       HMR_0736
UniprotID       SMS2_HUMAN', 'SMS1_HUMAN'
ReactionRule             Q8NHU3 or Q86VZ5
Name: 3306, dtype: object


ReactionID                                               HMR_0758
UniprotID       CERS4_HUMAN', 'CERS5_HUMAN', 'CERS2_HUMAN', 'C...
ReactionRule    Q9HA82 or Q8N5B7 or Q96G23 or Q8IU89 or Q6ZMG9...
Name: 3315, dtype: object


ReactionID                                               HMR_0588
UniprotID       ABHD5_HUMAN', 'PLCD_HUMAN', 'PLCE_HUMAN', 'LCL...
ReactionRule    Q8WTS1 or Q9NRZ5 or Q6ZWT7 or Q9NUQ2 or Q9NRZ7...
Name: 3460, dtype: object


Re

ReactionID          HMR_6664
UniprotID       CP26B_HUMAN'
ReactionRule          Q9NR63
Name: 4672, dtype: object


ReactionID          HMR_6665
UniprotID       CP26B_HUMAN'
ReactionRule          Q9NR63
Name: 4673, dtype: object


ReactionID                                               HMR_6668
UniprotID       UD2A3_HUMAN', 'UD3A1_HUMAN', 'UD15_HUMAN', 'UD...
ReactionRule    P36537 or Q6UWM9 or Q9BY64 or Q6NUS8 or P06133...
Name: 4676, dtype: object


ReactionID                                               HMR_6669
UniprotID       UD2A3_HUMAN', 'UD3A1_HUMAN', 'UD15_HUMAN', 'UD...
ReactionRule    P36537 or Q6UWM9 or Q9BY64 or Q6NUS8 or P06133...
Name: 4677, dtype: object


ReactionID      HMR_6673
UniprotID            NaN
ReactionRule         NaN
Name: 4681, dtype: object


ReactionID          HMR_6677
UniprotID       CP26A_HUMAN'
ReactionRule          O43174
Name: 4685, dtype: object


ReactionID          HMR_6678
UniprotID       CP26A_HUMAN'
ReactionRule          O43174
Name: 4686, d

...

ReactionID      HMR_9159
UniprotID            NaN
ReactionRule         NaN
Name: 7831, dtype: object


ReactionID      HMR_9167
UniprotID            NaN
ReactionRule         NaN
Name: 7839, dtype: object


ReactionID      HMR_9168
UniprotID            NaN
ReactionRule         NaN
Name: 7840, dtype: object


ReactionID      HMR_9204
UniprotID            NaN
ReactionRule         NaN
Name: 7847, dtype: object


ReactionID      HMR_9215
UniprotID            NaN
ReactionRule         NaN
Name: 7858, dtype: object


ReactionID      HMR_9219
UniprotID            NaN
ReactionRule         NaN
Name: 7862, dtype: object


ReactionID      HMR_9232
UniprotID            NaN
ReactionRule         NaN
Name: 7875, dtype: object


ReactionID      HMR_9234
UniprotID            NaN
ReactionRule         NaN
Name: 7877, dtype: object


ReactionID      HMR_9242
UniprotID            NaN
ReactionRule         NaN
Name: 7885, dtype: object


ReactionID      HMR_9253
UniprotID            NaN
ReactionRule         Na

ReactionID      PCRNtc
UniprotID          NaN
ReactionRule       NaN
Name: 8547, dtype: object


ReactionID           PCRNtm
UniprotID       MCAT_HUMAN'
ReactionRule         O43772
Name: 8548, dtype: object


ReactionID      PEt
UniprotID       NaN
ReactionRule    NaN
Name: 8554, dtype: object


ReactionID             PGPPT
UniprotID       PGPS1_HUMAN'
ReactionRule          Q32NB8
Name: 8559, dtype: object


ReactionID      PSt3
UniprotID        NaN
ReactionRule     NaN
Name: 8583, dtype: object


ReactionID         UGT1A10r
UniprotID       UD18_HUMAN'
ReactionRule         Q9HAW9
Name: 8631, dtype: object


ReactionID      EX_ahdt[e]
UniprotID              NaN
ReactionRule           NaN
Name: 8647, dtype: object


ReactionID      EX_dha[e]
UniprotID             NaN
ReactionRule          NaN
Name: 8651, dtype: object


ReactionID      EX_dtmp[e]
UniprotID              NaN
ReactionRule           NaN
Name: 8653, dtype: object


ReactionID      EX_dttp[e]
UniprotID              NaN
Reactio

ReactionID            RPEc
UniprotID       RPE_HUMAN'
ReactionRule        Q96AT9
Name: 9775, dtype: object


ReactionID      TAURCHAe
UniprotID            NaN
ReactionRule         NaN
Name: 9778, dtype: object


ReactionID      q10tm
UniprotID         NaN
ReactionRule      NaN
Name: 9781, dtype: object


ReactionID      AHCYSte
UniprotID           NaN
ReactionRule        NaN
Name: 9789, dtype: object


ReactionID      CBASPte
UniprotID           NaN
ReactionRule        NaN
Name: 9792, dtype: object


ReactionID      EX_ahcys[e]
UniprotID               NaN
ReactionRule            NaN
Name: 9799, dtype: object


ReactionID      EX_cbasp[e]
UniprotID               NaN
ReactionRule            NaN
Name: 9802, dtype: object


ReactionID      MAL_Lte
UniprotID           NaN
ReactionRule        NaN
Name: 9803, dtype: object


ReactionID          OROTGLUt
UniprotID       S22A7_HUMAN'
ReactionRule          Q9Y694
Name: 9804, dtype: object


ReactionID      PEPtr
UniprotID         NaN
ReactionRul

ReactionID      EX_leuasnasp[e]
UniprotID                   NaN
ReactionRule                NaN
Name: 10500, dtype: object


ReactionID      EX_leuleutrp[e]
UniprotID                   NaN
ReactionRule                NaN
Name: 10502, dtype: object


ReactionID      EX_propropro[e]
UniprotID                   NaN
ReactionRule                NaN
Name: 10558, dtype: object


ReactionID      EX_thrthrarg[e]
UniprotID                   NaN
ReactionRule                NaN
Name: 10578, dtype: object


ReactionID      EX_trptyrtyr[e]
UniprotID                   NaN
ReactionRule                NaN
Name: 10607, dtype: object


ReactionID      EX_tyralaphe[e]
UniprotID                   NaN
ReactionRule                NaN
Name: 10610, dtype: object


ReactionID      EX_valtrpval[e]
UniprotID                   NaN
ReactionRule                NaN
Name: 10631, dtype: object


ReactionID                       ALAARGGLYt
UniprotID       S15A1_HUMAN', 'S15A2_HUMAN'
ReactionRule               P46059 or 

ReactionID      4OHBUTm
UniprotID           NaN
ReactionRule        NaN
Name: 11462, dtype: object


ReactionID      4OHBUTtmc
UniprotID             NaN
ReactionRule          NaN
Name: 11463, dtype: object


ReactionID      4OHBUTtce
UniprotID             NaN
ReactionRule          NaN
Name: 11464, dtype: object


ReactionID      EX_4ohbut[e]
UniprotID                NaN
ReactionRule             NaN
Name: 11465, dtype: object


ReactionID      PAIL_hs_t1e
UniprotID               NaN
ReactionRule            NaN
Name: 11546, dtype: object


ReactionID                                        MLTHFte3
UniprotID       FOLR1_HUMAN', 'FOLR2_HUMAN', 'FOLR3_HUMAN'
ReactionRule                    P15328 or P41439 or P14207
Name: 11697, dtype: object


ReactionID                                  1A25DHVITD3te
UniprotID       CD36_HUMAN', 'SCRB1_HUMAN', 'NPCL1_HUMAN'
ReactionRule                   Q9UHC9 or Q8WTV0 or P16671
Name: 11712, dtype: object


ReactionID      SUCSALtm
UniprotID            N

In [16]:
#Converting filtered model list into DF. Now have a DF of Human1 model, filtered according to core/non-zero flux reactions.
filtered_model_df = pd.DataFrame(filtered_model,columns=['ReactionID','UniprotID','ReactionRule'])
filtered_model_df

,ReactionID,UniprotID,ReactionRule
13,HMR_4355,"ALDOA_HUMAN', 'ALDOB_HUMAN', 'ALDOC_HUMAN'",P09972 or P05062 or P04075 or P04075
16,HMR_4363,"ENOA_HUMAN', 'ENOB_HUMAN', 'ENOG_HUMAN'",P06733 or P13929 or P09104
17,HMR_4365,"PMGE_HUMAN', 'PGAM1_HUMAN', 'PGAM4_HUMAN', 'PG...",P15259 or P18669 or P07738 or Q8N0Y7
18,HMR_4368,"PGK2_HUMAN', 'CRIS3_HUMAN', 'PGK1_HUMAN', 'TGO...",P54108 or P00558 or Q5JRA6 or P07205
20,HMR_4371,"PMGE_HUMAN', 'PGAM1_HUMAN', 'PGAM4_HUMAN', 'PG...",P15259 or P18669 or P07738 or Q8N0Y7
...,...,...,...
13024,HMR_10062,NaN,NaN
13025,HMR_10063,NaN,NaN
13026,HMR_10064,NaN,NaN
13027,HMR_10065,NaN,NaN


In [ ]:
#Now need to compare this model to our Myc RKO dataset and find the coverage. Ideally will cover majority of core reactions. 